In [20]:
import pandas as pd
import re

ImportError: attempted relative import with no known parent package

In [2]:
df = pd.read_excel("network.xlsx",
									 header=None,
									 names = ["first", "middle", "last"]).fillna("")

df['title'] = df.apply(lambda row: ' '.join(row), axis=1)

In [3]:
with open("rob-top2000-labels.txt", "w") as f:
	f.write("\n".join(df['title']))

labelled_edges = []

for i, src in df.iterrows():
	linking_word = src['last']
	targets = df[df['first'] == linking_word]
	for j, tar in targets.iterrows():
		labelled_edges.append(((i, src['title']), (j, tar['title'])))

with open("rob-top2000-graph.txt", "w") as f:
	f.write(str(len(df['title'])) + "\n")

	for (i, _), (j, _) in labelled_edges:
		f.write(f'{i} {j}\n')

with open("rob-top2000-labelled-edges.txt", "w") as f:
	for (i, src_title), (j, tar_title) in labelled_edges:
		f.write(f"{src_title} -> {tar_title} ({i} -> {j})\n")

In [12]:
undirected_edges = {frozenset([i, j]) for (i, _), (j, _) in labelled_edges[:1000]}
undirected_edges_list = [tuple(e) for e in undirected_edges if len(e) > 1]

directed_undirected_edges_list = undirected_edges_list + [(t, s) for (s, t) in undirected_edges_list]

len(labelled_edges), len(undirected_edges), len(undirected_edges_list), len(directed_undirected_edges_list)

(3438, 1000, 991, 1982)

In [13]:
with open("rob-top2000-KALP.dimacs", "w") as f:
    f.write(f"p sp {len(df["title"])} {len(directed_undirected_edges_list)}\n")

    for s, t in directed_undirected_edges_list:
        f.write(f"a {s + 1} {t + 1} 1\n")

In [9]:
import subprocess

with open("rob-top2000-graph.txt", "r") as f:
	result = subprocess.run(['../../lpath','DFBNB'], stdin=f, stdout=subprocess.PIPE, text=True)

path = list(map(int, re.search(r'Longest path: (.+)\s', result.stdout).group(1).split()))
titles = [df.iloc[i]['title'] for i in path]

print(result.stdout)
print('\n'.join(titles))

lpath: neighbours_graph.c:149: verify_path: Assertion `path->count > 0' failed.


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
import subprocess

with open("top2000-graph.txt", "r") as f:
	result = subprocess.run(['../../lpath','DFBNB'], stdin=f, stdout=subprocess.PIPE, text=True)

print(result.stdout)


# Depth first Branch and Bound takes longer